<a href="https://colab.research.google.com/github/SuchitraShankar07/ML-Hackathon/blob/master/ML_HACKATHON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================================
# ENHANCED SCRIPT — Maximum Accuracy Hybrid HMM + Advanced Heuristics
# ============================================================================
# Part 1: Importing
import numpy as np
import pandas as pd
from collections import defaultdict, Counter
import dill as pickle
import random
from typing import List, Dict, Tuple, Set
import json
from tqdm import tqdm
import warnings
import re
import string
import time

warnings.filterwarnings('ignore')
np.random.seed(42)
random.seed(42)

# ============================================================================
# Part 2: Data Loading and Preprocessing
# ============================================================================

def load_corpus(filename='corpus.txt'):
    """Load and preprocess the word corpus (lowercase, a-z only)."""
    with open(filename, 'r', encoding='utf-8', errors='ignore') as f:
        words = []
        for line in f:
            w = line.strip().lower()
            if not w:
                continue
            w = re.sub(r'[^a-z]', '', w)
            if 1 <= len(w) <= 20:
                words.append(w)
    print(f"Loaded {len(words)} cleaned words from {filename}")
    return words


def load_test_words(filename='test.txt'):
    """Load test words exactly as they appear (no preprocessing)."""
    with open(filename, 'r', encoding='utf-8', errors='ignore') as f:
        words = [line.strip() for line in f if line.strip()]
    print(f"Loaded {len(words)} test words from {filename}")
    return words

# ============================================================================
# Part 3: Enhanced HMM with Advanced Pattern Recognition
# ============================================================================

class HangmanHMM:
    """
    Enhanced model with:
      - Positional emission probabilities with adaptive smoothing
      - Word frequency weighting
      - Advanced n-gram modeling (bigrams, trigrams, 4-grams)
      - Common letter pattern detection
      - Double letter detection
    """
    def __init__(self):
        self.models = {}
        self.alphabet = 'abcdefghijklmnopqrstuvwxyz'
        self.letter_to_idx = {c:i for i,c in enumerate(self.alphabet)}
        self.idx_to_letter = {i:c for i,c in enumerate(self.alphabet)}
        self.bigram_freq = defaultdict(Counter)
        self.trigram_freq = defaultdict(Counter)
        self.fourgram_freq = defaultdict(Counter)
        self.overall_freq = Counter()
        self.word_freq = Counter()
        self.double_letter_freq = Counter()
        self.common_endings = defaultdict(Counter)
        self.common_beginnings = defaultdict(Counter)

    def train(self, words: List[str]):
        """Train HMM models with enhanced pattern recognition."""
        print("Training Enhanced HMM models...")

        # Count word frequencies for weighting
        for word in words:
            self.word_freq[word] += 1

        # Group words by length
        words_by_length = defaultdict(list)
        for word in words:
            words_by_length[len(word)].append(word)

            # Learn n-gram patterns with frequency weighting
            freq_weight = np.log1p(self.word_freq[word])
            for i, letter in enumerate(word):
                self.overall_freq[letter] += freq_weight

                # Detect double letters
                if i > 0 and word[i-1] == letter:
                    self.double_letter_freq[letter] += freq_weight

                # N-grams
                if i > 0:
                    self.bigram_freq[word[i - 1]][letter] += freq_weight
                if i > 1:
                    bigram_key = word[i - 2 : i]
                    self.trigram_freq[bigram_key][letter] += freq_weight
                if i > 2:
                    trigram_key = word[i - 3 : i]
                    self.fourgram_freq[trigram_key][letter] += freq_weight

                # Common patterns
                if i < 3:
                    self.common_beginnings[i][letter] += freq_weight
                if i >= len(word) - 3:
                    self.common_endings[len(word) - i - 1][letter] += freq_weight

        # Train one emission model per word length
        for length, word_list in tqdm(sorted(words_by_length.items()), desc="Training HMMs"):
            if len(word_list) < 3:
                continue

            # Emission probabilities with frequency weighting
            emission_counts = np.zeros((length, 26))
            start_letters = Counter()
            end_letters = Counter()

            for word in word_list:
                freq_weight = np.log1p(self.word_freq[word])
                start_letters[word[0]] += freq_weight
                end_letters[word[-1]] += freq_weight

                for pos, letter in enumerate(word):
                    if letter in self.letter_to_idx:
                        emission_counts[pos, self.letter_to_idx[letter]] += freq_weight

            # Adaptive smoothing based on data size
            smoothing = max(0.1, 1.0 / np.sqrt(len(word_list)))
            emission_counts += smoothing
            row_sums = emission_counts.sum(axis=1, keepdims=True)
            row_sums[row_sums == 0] = 1.0
            emission_probs = emission_counts / row_sums

            # Store model
            self.models[length] = {
                "emission_probs": emission_probs,
                "word_list": word_list,
                "start_letters": start_letters,
                "end_letters": end_letters,
                "word_set": set(word_list),
            }

        print(f"Trained HMMs for {len(self.models)} word lengths")
        print(f"Learned {len(self.bigram_freq)} bigrams, {len(self.trigram_freq)} trigrams, {len(self.fourgram_freq)} 4-grams")

    def get_matching_words(self, masked_word: str, guessed_letters: Set[str]) -> List[str]:
        L = len(masked_word)
        if L not in self.models:
            return []
        word_list = self.models[L]['word_list']
        pattern = '^' + re.escape(masked_word).replace('\\_', '.') + '$'
        regex = re.compile(pattern)
        matching = []
        for w in word_list:
            if not regex.match(w):
                continue
            ok = True
            for i,(m,ch) in enumerate(zip(masked_word, w)):
                if m == '_' and ch in guessed_letters:
                    ok = False
                    break
                if m != '_' and m != ch:
                    ok = False
                    break
            if ok:
                matching.append(w)
        return matching

    def get_letter_probabilities(self, masked_word: str, guessed_letters: Set[str]) -> Dict[str,float]:
        """Enhanced probability calculation with multiple signals."""
        L = len(masked_word)
        if L not in self.models:
            total = sum(self.overall_freq.values()) or 1
            probs = {c: self.overall_freq.get(c,0)/total for c in self.alphabet}
            for g in guessed_letters:
                probs[g] = 0.0
            s = sum(probs.values()) or 1.0
            return {c: probs[c]/s for c in probs}

        model = self.models[L]
        matching = self.get_matching_words(masked_word, guessed_letters)
        counts = np.zeros(26)

        if matching:
            # Frequency-weighted candidate analysis
            for w in matching:
                freq_weight = np.log1p(self.word_freq.get(w, 1))
                for i,ch in enumerate(w):
                    if masked_word[i] == '_' and ch in self.letter_to_idx:
                        counts[self.letter_to_idx[ch]] += freq_weight
        else:
            # Multi-signal positional approach
            emission = model['emission_probs']

            # Base positional probabilities
            for i,ch in enumerate(masked_word):
                if ch == '_' and i < emission.shape[0]:
                    counts += emission[i] * 2.0  # Increased weight

            # Advanced n-gram boosting
            for i,ch in enumerate(masked_word):
                if ch != '_':
                    continue

                # 4-gram boost (strongest signal)
                if i > 2:
                    if all(masked_word[j] != '_' for j in range(i-3, i)):
                        trigram_key = masked_word[i-3:i]
                        for letter, c in self.fourgram_freq.get(trigram_key, {}).items():
                            if letter in self.letter_to_idx:
                                counts[self.letter_to_idx[letter]] += c * 0.35

                # Trigram boost
                if i > 1:
                    if masked_word[i-1] != '_' and masked_word[i-2] != '_':
                        bg = masked_word[i-2:i]
                        for letter, c in self.trigram_freq.get(bg, {}).items():
                            if letter in self.letter_to_idx:
                                counts[self.letter_to_idx[letter]] += c * 0.25

                # Bigram boost (both directions)
                if i > 0 and masked_word[i-1] != '_':
                    prev = masked_word[i-1]
                    for letter, c in self.bigram_freq.get(prev, {}).items():
                        if letter in self.letter_to_idx:
                            counts[self.letter_to_idx[letter]] += c * 0.15

                if i < len(masked_word) - 1 and masked_word[i+1] != '_':
                    next_char = masked_word[i+1]
                    for letter, c in self.bigram_freq.items():
                        if next_char in c and letter in self.letter_to_idx:
                            counts[self.letter_to_idx[letter]] += c[next_char] * 0.12

                # Double letter boost
                if i > 0 and masked_word[i-1] != '_':
                    prev = masked_word[i-1]
                    if prev in self.letter_to_idx:
                        counts[self.letter_to_idx[prev]] += self.double_letter_freq.get(prev, 0) * 0.2

            # Position-specific patterns
            for i, ch in enumerate(masked_word):
                if ch == '_':
                    if i < 3:
                        for letter, c in self.common_beginnings.get(i, {}).items():
                            if letter in self.letter_to_idx:
                                counts[self.letter_to_idx[letter]] += c * 0.08
                    if i >= len(masked_word) - 3:
                        pos_from_end = len(masked_word) - i - 1
                        for letter, c in self.common_endings.get(pos_from_end, {}).items():
                            if letter in self.letter_to_idx:
                                counts[self.letter_to_idx[letter]] += c * 0.08

        # Zero out guessed letters
        for g in guessed_letters:
            if g in self.letter_to_idx:
                counts[self.letter_to_idx[g]] = 0.0

        s = counts.sum()
        if s <= 0:
            total = sum(self.overall_freq.values()) or 1
            probs = {c: self.overall_freq.get(c,0)/total for c in self.alphabet}
            for g in guessed_letters:
                probs[g] = 0.0
            ss = sum(probs.values()) or 1.0
            return {c: probs[c]/ss for c in probs}

        probs = counts / s
        return {self.idx_to_letter[i]: float(probs[i]) for i in range(26)}

# ============================================================================
# Part 4: Hangman Game Environment
# ============================================================================

class HangmanEnvironment:
    def __init__(self, word: str, max_lives: int = 6):
        self.word = word.lower()
        self.max_lives = max_lives
        self.reset()

    def reset(self):
        self.lives = self.max_lives
        self.guessed_letters = set()
        self.masked_word = '_' * len(self.word)
        self.wrong_guesses = 0
        self.repeated_guesses = 0
        self.done = False
        self.won = False
        self.num_revealed = 0
        return self.get_state()

    def get_state(self) -> Dict:
        return {
            'masked_word': self.masked_word,
            'guessed_letters': set(self.guessed_letters),
            'lives': self.lives,
            'wrong_guesses': self.wrong_guesses,
            'repeated_guesses': self.repeated_guesses,
            'done': self.done,
            'won': self.won,
            'num_revealed': self.num_revealed,
            'progress': (self.num_revealed / len(self.word)) if len(self.word)>0 else 0.0
        }

    def step(self, letter: str) -> Tuple[Dict, float, bool]:
        letter = letter.lower()
        if letter in self.guessed_letters:
            self.repeated_guesses += 1
            self.lives -= 1
            if self.lives <= 0:
                self.done = True
                self.won = False
            return self.get_state(), -2.0, self.done

        self.guessed_letters.add(letter)
        if letter in self.word:
            new_mask = list(self.masked_word)
            revealed = 0
            for i,ch in enumerate(self.word):
                if ch == letter and new_mask[i] == '_':
                    new_mask[i] = letter
                    revealed += 1
            self.masked_word = ''.join(new_mask)
            self.num_revealed += revealed
            reward = 5.0 + (8.0 * revealed)
            if '_' not in self.masked_word:
                self.done = True
                self.won = True
                reward += 80.0 + (10.0 * self.lives)
        else:
            self.lives -= 1
            self.wrong_guesses += 1
            reward = -8.0
            if self.lives <= 0:
                self.done = True
                self.won = False
                reward -= 40.0
        return self.get_state(), reward, self.done

# ============================================================================
# Part 5: Enhanced Hybrid Agent with Smart Heuristics
# ============================================================================

class HybridHangmanAgent:
    """
    Enhanced decision strategy:
     - Smarter candidate filtering
     - Frequency-weighted selection
     - Context-aware disambiguation
     - Minimal RL for edge cases
    """
    def __init__(self, hmm_model: HangmanHMM,
                 epsilon_rl: float = 0.02,
                 epsilon_decay: float = 0.99997,
                 epsilon_min: float = 0.001,
                 lr: float = 0.1, gamma: float = 0.96):
        self.hmm = hmm_model
        self.epsilon_rl = epsilon_rl
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min
        self.lr = lr
        self.gamma = gamma
        self.alphabet = 'abcdefghijklmnopqrstuvwxyz'
        self.q_table = defaultdict(lambda: defaultdict(float))

    def state_key(self, state: Dict) -> str:
        mk = state['masked_word']
        guessed = ''.join(sorted(state['guessed_letters']))
        lives = state['lives']
        prog = int(state['progress']*10)
        return f"{mk}|{guessed}|{lives}|{prog}"

    def choose_action(self, state: Dict, training: bool = True) -> str:
        available = [c for c in self.alphabet if c not in state['guessed_letters']]
        if not available:
            return random.choice(list(self.alphabet))

        # Very minimal RL exploration
        if training and random.random() < self.epsilon_rl:
            return random.choice(available)

        masked = state['masked_word']
        guessed = set(state['guessed_letters'])
        matching = self.hmm.get_matching_words(masked, guessed)

        # Single candidate: deterministic play
        if len(matching) == 1:
            cand = matching[0]
            for ch in cand:
                if ch not in guessed:
                    return ch

        # Small candidate set: frequency-weighted smart selection
        if 1 < len(matching) <= 15:
            freq = Counter()
            for w in matching:
                word_weight = np.log1p(self.hmm.word_freq.get(w, 1))
                for i, ch in enumerate(w):
                    if masked[i] == '_' and ch not in guessed:
                        freq[ch] += word_weight

            if freq:
                # Pick letter that disambiguates most effectively
                best = freq.most_common(1)[0][0]
                return best

        # Fallback: use enhanced HMM probabilities
        probs = self.hmm.get_letter_probabilities(masked, guessed)
        best_letter = max(available, key=lambda c: probs.get(c, 0.0))
        return best_letter

    def update(self, state: Dict, action: str, reward: float, next_state: Dict, done: bool):
        s = self.state_key(state)
        ns = self.state_key(next_state)
        cur = self.q_table[s].get(action, 0.0)
        if done:
            nxt_max = 0.0
        else:
            av = [a for a in self.alphabet if a not in next_state['guessed_letters']]
            nxt_max = max([self.q_table[ns].get(a,0.0) for a in av], default=0.0)
        td = reward + self.gamma * nxt_max - cur
        self.q_table[s][action] = cur + self.lr * td
        if self.epsilon_rl > self.epsilon_min:
            self.epsilon_rl *= self.epsilon_decay

# ============================================================================
# Part 6: Training Pipeline
# ============================================================================

def train_agent(agent: HybridHangmanAgent, train_words: List[str], num_episodes: int = 25000):
    print(f"Training hybrid agent for {num_episodes} episodes")
    sorted_words = sorted(train_words, key=len)
    curriculum_size = max(1, len(sorted_words)//4)
    episode_rewards = []
    start = time.time()

    for ep in tqdm(range(num_episodes), desc="Training"):
        if ep < num_episodes//4:
            word = random.choice(sorted_words[:curriculum_size])
        elif ep < num_episodes//2:
            word = random.choice(sorted_words[:min(len(sorted_words), curriculum_size*2)])
        else:
            word = random.choice(train_words)

        env = HangmanEnvironment(word)
        state = env.reset()
        total_reward = 0.0
        steps = 0

        while not state['done']:
            action = agent.choose_action(state, training=True)
            next_state, reward, done = env.step(action)
            agent.update(state, action, reward, next_state, done)
            state = next_state
            total_reward += reward
            steps += 1
            if steps > 26:
                break

        episode_rewards.append(total_reward)

        if (ep+1) % 5000 == 0:
            elapsed = time.time() - start
            avg_recent = np.mean(episode_rewards[-5000:])
            print(f"Ep {ep+1}/{num_episodes}, avg_reward= {avg_recent:.2f}, eps_rl={agent.epsilon_rl:.4f}, time={elapsed:.1f}s")

    return episode_rewards

# ============================================================================
# Part 7: Evaluation Pipeline
# ============================================================================

def evaluate_agent(agent: HybridHangmanAgent, test_words: List[str], num_games: int = 2000, verbose_sample: int = 100):
    wins = 0
    total_wrong = 0
    total_repeated = 0
    game_details = []
    test_sample = [test_words[i % len(test_words)] for i in range(num_games)]

    for i, w in enumerate(tqdm(test_sample, desc="Evaluating")):
        env = HangmanEnvironment(w)
        state = env.reset()
        guesses = []
        per_step_log = []

        while not state['done']:
            action = agent.choose_action(state, training=False)
            guesses.append(action)
            next_state, _, _ = env.step(action)
            if i < verbose_sample:
                per_step_log.append({
                    'guess': action,
                    'masked': next_state['masked_word'],
                    'lives': next_state['lives']
                })
            state = next_state

        if state['won']:
            wins += 1
        total_wrong += state['wrong_guesses']
        total_repeated += state['repeated_guesses']

        if i < verbose_sample:
            game_details.append({
                'word': w,
                'won': state['won'],
                'wrong': state['wrong_guesses'],
                'repeated': state['repeated_guesses'],
                'guesses': guesses,
                'log': per_step_log
            })

    success_rate = wins / num_games
    final_score = (success_rate * num_games) - (total_wrong * 5) - (total_repeated * 2)

    results = {
        'num_games': num_games,
        'wins': wins,
        'success_rate': success_rate,
        'total_wrong_guesses': total_wrong,
        'total_repeated_guesses': total_repeated,
        'final_score': final_score,
        'sample_games': game_details
    }

    print("="*70)
    print("EVALUATION SUMMARY".center(70))
    print("="*70)
    print(f"Games: {num_games}, Wins: {wins}, Success Rate: {success_rate:.2%}")
    print(f"Total wrong: {total_wrong}, Total repeated: {total_repeated}")
    print(f"Final Score: {final_score:.2f}")
    print("="*70)

    return results

# ============================================================================
# Part 8: Main Execution
# ============================================================================

def main(num_train_episodes=25000, eval_games=2000):
    t0 = time.time()
    print("Loading data...")
    train_words = load_corpus('corpus.txt')
    test_words = load_test_words('test.txt')

    print("Training Enhanced HMM...")
    hmm = HangmanHMM()
    hmm.train(train_words)

    print("Initializing enhanced agent...")
    agent = HybridHangmanAgent(hmm_model=hmm,
                               epsilon_rl=0.03,
                               epsilon_decay=0.99997,
                               epsilon_min=0.001,
                               lr=0.1, gamma=0.96)

    print("Training agent...")
    episode_rewards = train_agent(agent, train_words, num_episodes=num_train_episodes)

    print("Evaluating agent...")
    results = evaluate_agent(agent, test_words, num_games=eval_games, verbose_sample=100)

    with open('trained_hybrid_agent.pkl', 'wb') as f:
        pickle.dump({'agent': agent, 'hmm': hmm}, f)
    with open('eval_results.json', 'w') as f:
        json.dump(results, f, indent=2)

    t1 = time.time()
    print(f"Total runtime: {t1 - t0:.1f}s")
    return agent, hmm, results, episode_rewards

# ============================================================================
# Part 9: Run Everything
# ============================================================================

if __name__ == "__main__":
    agent, hmm, results, episode_rewards = main(num_train_episodes=40000, eval_games=2000)

Loading data...
Loaded 49972 cleaned words from corpus.txt
Loaded 2000 test words from test.txt
Training Enhanced HMM...
Training Enhanced HMM models...


Training HMMs: 100%|██████████| 20/20 [00:00<00:00, 58.49it/s]


Trained HMMs for 20 word lengths
Learned 26 bigrams, 674 trigrams, 7782 4-grams
Initializing enhanced agent...
Training agent...
Training hybrid agent for 40000 episodes


Training:  13%|█▎        | 5006/40000 [01:39<11:15, 51.79it/s]

Ep 5000/40000, avg_reward= -10.76, eps_rl=0.0078, time=99.0s


...

Ep 35000/40000, avg_reward= 54.73, eps_rl=0.0010, time=988.2s


Training: 100%|██████████| 40000/40000 [19:05<00:00, 34.91it/s]


Ep 40000/40000, avg_reward= 53.99, eps_rl=0.0010, time=1145.8s
Evaluating agent...


Evaluating: 100%|██████████| 2000/2000 [01:02<00:00, 32.04it/s]


                          EVALUATION SUMMARY                          
Games: 2000, Wins: 636, Success Rate: 31.80%
Total wrong: 10468, Total repeated: 0
Final Score: -51704.00
Total runtime: 1215.8s
